In [1]:
import pickle
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
from sklearn import preprocessing

from sklearn.preprocessing import OneHotEncoder as ohc

def sliding_window(data,stepSize,windowSize):
    for x in range(0,len(data)-windowSize+1,stepSize):
        yield(x,data[x:x+windowSize])

def pred_window(clf,data):
    test_set=[]
    for(x,window) in sliding_window(data,3,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    #print(np.shape(test_set))
    return clf.predict(test_set)

def test_set(data):
    test_set=[]
    for(x,window) in sliding_window(data,3,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    return (test_set)

#filename queue
filename_queue=glob.glob('./featuredata/compfeature[1-2].csv')
data=list()
context_label=list()
context_label_count=np.zeros(4)

for filename in filename_queue:
    print(filename)
    file=open(filename, newline='')
    reader=csv.reader(file)
    #context=reader[seq(1, nrow(reader), 2),]
    #data = reader[seq(1, nrow(reader), 2),]
    i=0
    array=list()


    for row in reader:
        i+=1
        if(int(i)%2==1):
            context_label.append(row)
            context_label_count[int(row[0])]+=1
        else:
            array=np.array(row)
            array=array.reshape(600,6,5)
            data.append(array)


./featuredata/compfeature2.csv
./featuredata/compfeature1.csv


In [2]:
# load svm
filename='./svm.sav'
svm = pickle.load(open(filename, 'rb'))

# load knn
filename='./knn.sav'
knn = pickle.load(open(filename, 'rb'))

# load rf
filename='./rf.sav'
rf = pickle.load(open(filename, 'rb'))

# load ETC
filename='./ETC.sav'
ETC = pickle.load(open(filename, 'rb'))

In [ ]:
"""
이거는 그냥 볼려고 해본거구 무시하셈
pred=[]
for x in data: #10 minute granularity
    pred.append(pred_window(ETC,x))
print(context_label_count)
print(context_label)
print(pred)
"""

0 
1
2
3
4
5
6
7
8

In [3]:
###이게 ETC complex prediction
pred=list()
pred2=[] ##이게 prediction
test_pred_proba=[]
for x in data: #10 minute granularity
    test_pred_proba.append(ETC.predict_proba(preprocessing.quantile_transform(test_set(x))))
print(np.shape(test_pred_proba))
for i in test_pred_proba:
    for j in i:
        if np.max(j)<0.3: #확률 0.3미만이면 idle
            pred.append(16)
        else:
            pred.append(np.argmax(j))

for i in range(int(len(pred)/199)):
    pred2.append(pred[199*i:199*i+199])
for i in pred2:
    i.append(16)
pred2=np.array(pred2)
pred2=pred2.reshape(len(data),10,20)

print(context_label)
print(pred2)


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (199). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mo

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (199). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mo

(21, 199, 10)
[['1'], ['1'], ['2'], ['3'], ['1'], ['2'], ['1'], ['1'], ['1'], ['1'], ['2'], ['2'], ['3'], ['3'], ['1'], ['1'], ['2'], ['2'], ['1'], ['3'], ['3']]
[[[16 16 16 ... 16 16  0]
  [16 16 16 ...  8  8  8]
  [16  8  8 ...  7  7  7]
  ...
  [ 8 16 16 ...  3  3  3]
  [16  3 16 ...  8  8  2]
  [16  8 16 ...  8  8 16]]

 [[ 8 16  7 ... 16  3  3]
  [ 3  3 16 ... 16  8 16]
  [ 8  8 16 ... 16  8  8]
  ...
  [ 2 16 16 ... 16 16 16]
  [16  8 16 ...  7 16 16]
  [16  8  8 ... 16 16 16]]

 [[16  0  0 ...  8  8  8]
  [16  8  8 ... 16 16 16]
  [ 0  0  0 ...  8  8  8]
  ...
  [16  0  0 ...  8  8  8]
  [ 8  8  8 ... 16 16  8]
  [ 8  8  8 ... 16  8 16]]

 ...

 [[16 16 16 ... 16 16  8]
  [16  8 16 ...  8  2 16]
  [16  8 16 ...  8  8  3]
  ...
  [16  8 16 ... 16 16 16]
  [16 16  7 ... 16  8  8]
  [ 8 16 16 ... 16 16 16]]

 [[16 16  2 ...  8  8 16]
  [ 8 16 16 ...  8 16 16]
  [16  2  8 ...  8  8  8]
  ...
  [16  0  0 ... 16 16 16]
  [16 16  8 ...  8 16  8]
  [16  7  7 ... 16 16 16]]

 [[16 16  2 

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (199). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mo

In [4]:
### Context prediction 용 데이터 만들기 길이 17*10 = 170
context_data=[]
for context in pred2:
    context_=[]
    for minute in context:
        activity_count=np.zeros(17)
        for activity in minute:
            activity_count[activity]+=1
        context_.append(activity_count)
        context=list(context)
    context_data.append(context_)
context_data=np.array(context_data)
context_data=context_data.reshape(len(context_data),170)


In [ ]:
print(np.shape(context_data))
print(np.shape(context_label))
print(context_label)

In [ ]:
X=context_data
y=context_label


In [ ]:
### 여기서부터 context prediction 시작
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

clf=RandomForestClassifier()
clf.fit(X,y)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
scores=cross_val_score(clf,X,y,cv=3,scoring='accuracy')
print(scores.mean())

In [ ]:
from joblib import dump, load

filename='./complex_rf.sav'
pickle.dump(clf,open(filename,'wb'))
dump(clf,'complex_rf.joblib')

In [ ]:
clf=KNeighborsClassifier()
clf.fit(X,y)

In [ ]:

scores=cross_val_score(clf,X,y,cv=3,scoring='accuracy')
print(scores.mean())